In [1]:
from PIL import Image
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
import numpy as np
import torch

In [2]:
model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14").cuda()
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")
valid_dataset = load_dataset("danjacobellis/aria_ea_rgb_100k",split="validation")

Resolving data files:   0%|          | 0/70 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/70 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/18 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [3]:
classes = [
    "kitchen",
    "cooking",
    "food",
    "drink",
    "spill",
    "table",
    "television",
    "phone",
    "laptop",
    "video game",
    "board game",
    "clothes",
    "laundry",
]

In [4]:
def get_clip(sample):
    with torch.no_grad():
        image = sample['image']
        inputs = processor(text=classes, images=image, return_tensors="pt", padding=True)
        for k in inputs.keys():
            if hasattr(inputs[k], "device"):
                inputs[k] = inputs[k].cuda()
        outputs = model(**inputs)
        sample['clip_logit'] = outputs.logits_per_image
        return sample

In [ ]:
clip_dataset = valid_dataset.map(get_clip,writer_batch_size=256);

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]